In [1]:
# 강사님 코드
key = "sk-fHu2XOAEabWNOaTq35n2QmBKKAFU2mosRnsCMQz5OcT3BlbkFJNAr4-bIwJlm8MdKiwurinXpZ7oloiaTpxUUFKa0DUA"

In [2]:
import os

os.environ["OPENAI_API_KEY"] = key # 환경변수에 임시로 지정정

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature = 0.1)

In [5]:
result = llm.invoke("세종대왕이 누구인지 설명해 주세요")

In [6]:
type(result)

langchain_core.messages.ai.AIMessage

In [7]:
result.content

'세종대왕은 조선시대 4대 왕 중 하나로, 조선시대의 제4대 군주이자 문신이다. 세종대왕은 1418년에 태어나 1450년부터 1452년까지 조선의 왕위에 오르고 1418년부터 1450년까지 32년 동안 통치했다. 그는 조선시대의 가장 위대한 왕 중 한 명으로 꼽히며, 한글을 창제하고 과학기술, 문화, 예술 등을 발전시키는 데 큰 기여를 했다. 세종대왕은 또한 국방력을 강화하고 외국과의 관계를 조절하여 조선의 안정과 번영을 이룩하는 데 큰 역할을 했다.'

In [8]:
from langchain_core.prompts import PromptTemplate

In [9]:
template = "{who}가 누구인지 설명해주세요"
prompt = PromptTemplate(template = template, input_variables = ["who"])

lim_chain = prompt | llm 

In [10]:
result = lim_chain.invoke({"who": "안중근의사"})
result.content

'안중근(1879-1910)은 대한독립운동가이자 독립운동의 선봉장으로 활동한 인물입니다. 안중근은 일제 강점기에 일본의 식민지화 정책에 항거하고 대한독립을 위해 노력했습니다. 그는 1909년 하코다테에서 일본 총독 이토 히로부미를 암살한 사건으로 일본에 체포되어 처형당했습니다. 안중근의 희생은 대한독립운동가로서의 업적을 인정받아 오늘날에도 그의 이름은 대한민국 역사에서 큰 의미를 갖고 있습니다.'

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [12]:
template = """
당신은 휴먼 교육센터의 친절한 챗봇입니다.
이전 대화 : {history}
질문 : {input}
답변 : 
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ['history', 'input']
)

model = ChatOpenAI()
chain = prompt | model

In [20]:
session_store = {}
session_id = "oos"
if session_id not in session_store:
    session_store[session_id] = ChatMessageHistory()

session_history = session_store[session_id]

run_with_meassage_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: session_history,
    input_messages_key =  "input",
    history_messages_key= "history"
)

In [ ]:
# result = run_with_meassage_history.invoke(
#     {"input": "당신은 소속이 어디입니까?"},
#     config= { "configurabel": {"session_id": "oos"} }
#     )

In [23]:
result = run_with_meassage_history.invoke(
    {
        "input": "당신은 소속이 어디입니까?"
    },
    config={
        "configurable": {"session_id": "oos"}
    }
)


In [24]:
result.content

'저는 휴먼 교육센터에 소속된 친절한 챗봇입니다. 어떤 도움이 필요하신가요?'

In [32]:
result = run_with_meassage_history.invoke(
    {
        "input": "눈오는 크리스마스를 주제로 짧은 시를 하나 써주세요"
    },
    config={
        "configurable" : {"session_id": "oos"}
    }
)

In [33]:
result.content

'눈이 펑펑 오는 크리스마스,\n하얀 눈더러운 거리,\n따스한 불빛이 빛나는 밤,\n함께하는 소중한 시간.'

In [34]:
session_history.messages

[HumanMessage(content='당신은 소속이 어디입니까?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='저는 휴먼 교육센터에 소속된 친절한 챗봇입니다. 어떤 도움이 필요하신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 72, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5f954398-cf72-4095-9498-716741a23b8f-0', usage_metadata={'input_tokens': 72, 'output_tokens': 54, 'total_tokens': 126, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='눈오는 크리스마스를 주제로 짧은 시를 하나 써주세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='눈이 펑펑 오는 크리스마스,\n하얀 눈더러운 거리,\n따스한 불빛이 빛나는 